In [44]:
import pandas as pd

# pd.set_option('display.max_columns', None) # Exibe todas as colunas

caminho_do_arquivo = r"data\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')

# fltra as colunas que devme ser exibidas
colunas = ['Numero_da_Ocorrencia','Classificacao_da_Ocorrência','Data_da_Ocorrencia','Municipio','UF','Regiao','Nome_do_Fabricante']
df = df[colunas]

# reonmeia das colunas para retirar os acentos
df.rename(columns={'Classificacao_da_Ocorrência':'Classificacao_da_Ocorrencia'}, inplace=True)

df.head()




,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT
3,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT
4,7757,Incidente Grave,2018-03-18,TORRES,RS,Sul,PIPER AIRCRAFT


Filtra os ultmos 3 meses de forma precisa

In [45]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# obtem a data dos utlimos 3 meses de forma precisa
data_atual = datetime.now()
data_menos_3_meses = data_atual - relativedelta(months=12)
dt_final = data_menos_3_meses.date()



In [46]:


# Converte a data que servirá de filtro no dataFrame para date
df['Data_da_Ocorrencia'] = pd.to_datetime(df['Data_da_Ocorrencia'])
data_filtro_menos_3_m = pd.to_datetime(dt_final)

# Filtra pelo filtro de data
df = df[df['Data_da_Ocorrencia'] >= data_filtro_menos_3_m ] 

# Retorna o numero de linhas
df.shape



(814, 7)

In [47]:
from sqlalchemy import create_engine, Integer, String, Date, VARCHAR, text

# Parâmetros de conexão
dbname    = 'postgres'
user      = 'postgres'
password  = '12345'
host      = 'localhost'
port      = '5432'

# Connection string
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Create connection wiht sqlalchemy
engine = create_engine(connection_string)

table_name = "anac_sql"

# Atualiza os dados do ano atual
cursor = engine.connect()
delete = text(f'''
                DELETE FROM public.{table_name} WHERE "Data_da_Ocorrencia" >='{data_filtro_menos_3_m.date()}'
              ''')
cursor.execute(delete)
cursor.commit()

df.to_sql(table_name, engine, index=False, if_exists='append', dtype={
    'Numero_da_Ocorrencia': Integer,
    'Classificacao_da_Ocorrencia': VARCHAR(50),
    'Data_da_Ocorrencia': Date
})
# replace = sobrescreve toda a tabela
# append = adiciona os dados ao final da tabela

engine.dispose()
cursor.close()


Faz uma consulta aos dados utlizando SQLAchemy e faz export para JSON

In [55]:

from sqlalchemy import create_engine, Integer, String, Date, VARCHAR, text

# Parâmetros de conexão
dbname    = 'postgres'
user      = 'postgres'
password  = '12345'
host      = 'localhost'
port      = '5432'

# Connection string
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Create connection wiht sqlalchemy
engine = create_engine(connection_string)

query = """
select * from public.anac_sql
"""

df2 = pd.read_sql_query(query, engine)
df2.head(15)

pasta_destino_json = r"destino\uf.json"
df2.to_json(pasta_destino_json, orient="records", lines=True)

pasta_destino_excel = r"destino\uf.xlsx"
df2.to_excel(pasta_destino_excel, index=False, sheet_name="guia_da_planilha")

pasta_destino_csv = r"destino\uf.csv"
df2.to_csv(pasta_destino_csv, sep=",")

# Parquet - já vem no formato zip, compactado
# Ja vem com compression
pasta_destino_parquet = r"destino\uf.parquet"
df2.to_parquet(pasta_destino_parquet, index=False)
# df2.to_parquet(pasta_destino_parquet, index=False, engine="pyarrow")
# df2.to_parquet(pasta_destino_parquet, index=False, engine="fastparquet")

# Parquet particionado
pasta_destino_parquet_particionado = r"destino\uf_part.parquet"
colunas_particionamento = ['regiao'] # é possível particionar por mais de uma coluna. EX: região e uf
df2.to_parquet(pasta_destino_parquet_particionado, index=False,partition_cols=colunas_particionamento)
# df2.to_parquet(pasta_destino_parquet_particionado, index=False,partition_cols=colunas_particionamento, engine="pyarrow")
